In [1]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import cross_val_score, cross_val_predict

In [2]:
df_dados = pd.read_csv('Downloads/sf-crime/train/train.csv',delimiter=',')
df_dados = df_dados.sample(frac=1)
df_dados

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
654642,2006-01-28 03:00:00,WARRANTS,WARRANT ARREST,Saturday,CENTRAL,NONE,400 Block of BROADWAY ST,-122.405065,37.798013
275430,2011-08-13 12:00:00,SUSPICIOUS OCC,SUSPICIOUS OCCURRENCE,Saturday,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403405,37.775421
376667,2010-02-10 18:15:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,CENTRAL,NONE,600 Block of DAVIS ST,-122.398584,37.797986
233590,2012-03-24 10:35:00,ROBBERY,"ROBBERY, BODILY FORCE",Saturday,BAYVIEW,"ARREST, BOOKED",600 Block of 20TH ST,-122.388077,37.760575
805397,2003-12-24 14:45:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,RICHMOND,NONE,400 Block of JOHN F KENNEDY DR,-122.465239,37.772503
...,...,...,...,...,...,...,...,...,...
625474,2006-06-29 20:00:00,BURGLARY,"BURGLARY OF STORE, FORCIBLE ENTRY",Thursday,NORTHERN,NONE,500 Block of HAIGHT ST,-122.431268,37.771947
526570,2007-12-02 22:40:00,SUSPICIOUS OCC,SUSPICIOUS OCCURRENCE,Sunday,SOUTHERN,NONE,1000 Block of HOWARD ST,-122.408625,37.778658
808380,2003-12-09 09:17:00,OTHER OFFENSES,OBSCENE PHONE CALLS(S),Tuesday,NORTHERN,NONE,2100 Block of LARKIN ST,-122.420479,37.797571
868415,2003-02-18 21:30:00,BURGLARY,"BURGLARY, UNLAWFUL ENTRY",Tuesday,NORTHERN,NONE,1100 Block of EDDY ST,-122.425779,37.782319


In [3]:
df_dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 878049 entries, 654642 to 51837
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Dates       878049 non-null  object 
 1   Category    878049 non-null  object 
 2   Descript    878049 non-null  object 
 3   DayOfWeek   878049 non-null  object 
 4   PdDistrict  878049 non-null  object 
 5   Resolution  878049 non-null  object 
 6   Address     878049 non-null  object 
 7   X           878049 non-null  float64
 8   Y           878049 non-null  float64
dtypes: float64(2), object(7)
memory usage: 67.0+ MB


In [4]:
df_dados.describe()

,X,Y
count,878049.000000,878049.000000
mean,-122.422616,37.771020
std,0.030354,0.456893
min,-122.513642,37.707879
25%,-122.432952,37.752427
50%,-122.416420,37.775421
75%,-122.406959,37.784369
max,-120.500000,90.000000


In [5]:
target = df_dados['Category'].array
target


<PandasArray>
[          'WARRANTS',     'SUSPICIOUS OCC',      'LARCENY/THEFT',
            'ROBBERY',      'LARCENY/THEFT',      'LARCENY/THEFT',
      'LARCENY/THEFT',       'NON-CRIMINAL',     'OTHER OFFENSES',
      'LARCENY/THEFT',
 ...
           'BURGLARY',    'SECONDARY CODES', 'DISORDERLY CONDUCT',
            'RUNAWAY',          'VANDALISM',           'BURGLARY',
     'SUSPICIOUS OCC',     'OTHER OFFENSES',           'BURGLARY',
     'MISSING PERSON']
Length: 878049, dtype: object

In [6]:
df_features = df_dados.drop(columns=['Category','Descript'])

df_features

,Dates,DayOfWeek,PdDistrict,Resolution,Address,X,Y
654642,2006-01-28 03:00:00,Saturday,CENTRAL,NONE,400 Block of BROADWAY ST,-122.405065,37.798013
275430,2011-08-13 12:00:00,Saturday,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403405,37.775421
376667,2010-02-10 18:15:00,Wednesday,CENTRAL,NONE,600 Block of DAVIS ST,-122.398584,37.797986
233590,2012-03-24 10:35:00,Saturday,BAYVIEW,"ARREST, BOOKED",600 Block of 20TH ST,-122.388077,37.760575
805397,2003-12-24 14:45:00,Wednesday,RICHMOND,NONE,400 Block of JOHN F KENNEDY DR,-122.465239,37.772503
...,...,...,...,...,...,...,...
625474,2006-06-29 20:00:00,Thursday,NORTHERN,NONE,500 Block of HAIGHT ST,-122.431268,37.771947
526570,2007-12-02 22:40:00,Sunday,SOUTHERN,NONE,1000 Block of HOWARD ST,-122.408625,37.778658
808380,2003-12-09 09:17:00,Tuesday,NORTHERN,NONE,2100 Block of LARKIN ST,-122.420479,37.797571
868415,2003-02-18 21:30:00,Tuesday,NORTHERN,NONE,1100 Block of EDDY ST,-122.425779,37.782319


In [ ]:
sns.pairplot(df_dados, vars=['Dates', 'DayOfWeek', 'PdDistrict', 'Resolution', 'Address'])

plt.show()

In [ ]:
def getModel():
    return LinearRegression()
    #return RandomForestRegressor()

In [ ]:
scores = cross_val_score(getModel(), df_features, target, cv=10)

np.mean(scores)